<a href="https://colab.research.google.com/github/05satyam/large_language_models/blob/master/Simple_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
import torch
torch.cuda.is_available()

True

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

In [6]:
model=AutoModelForCausalLM.from_pretrained("bigscience/bloom-3b", torch_dtype=torch.float16, device_map='auto')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.01G [00:00<?, ?B/s]

In [15]:
tokenizer=AutoTokenizer.from_pretrained("bigscience/tokenizer")

tokenizer_config.json:   0%|          | 0.00/227 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [7]:
print(model)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 2560)
    (word_embeddings_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-29): 30 x BloomBlock(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=2560, out_features=10240, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=10240, out_features=2560, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
  )
 

In [31]:

for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [9]:
def print_trainable_parameters(model):
  trainable_pram=0
  all_pram=0
  for _, param in model.named_parameters():
    all_pram += param.numel()
    if param.requires_grad:
      trainable_pram+=param.numel()
  print(f"trainable params: {trainable_pram} || all params: {all_pram}")

## Breakdown of how the trainable parameters are obtained:




```


LoRA Configuration: The LoraConfig specifies a rank r=8 and lora_alpha=16. The rank (r) indicates the dimensionality of the low-rank matrices that will be introduced. lora_alpha is a scaling factor applied to the low-rank matrices during training, but it does not affect the number of parameters directly.

Targeted Modules: The configuration targets "query_key_value" modules within the Bloom model's self-attention mechanism. In a typical transformer model, the "query_key_value" linear layer projects input embeddings to a higher-dimensional space for calculating attention scores. For a Bloom model, this linear layer projects from 2560 dimensions to 7680 dimensions (in_features=2560, out_features=7680). Normally, this would result in a weight matrix of size 2560 x 7680.

Applying LoRA: Instead of adapting the entire weight matrix, LoRA introduces two smaller matrices, A and B, where A has dimensions 2560 x r and B has dimensions r x 7680. With r=8, the dimensions of A and B are 2560 x 8 and 8 x 7680, respectively.

Calculating Trainable Parameters:

For matrix A, the number of parameters is 2560 * 8 = 20,480.
For matrix B, the number of parameters is 8 * 7680 = 61,440.
Since LoRA is applied to the "query_key_value" projections in each of the 30 BloomBlock layers, the total number of parameters introduced by LoRA is (20,480 + 61,440) * 30 = 2,457,600.
Therefore, after applying the LoRA configuration with the specified parameters, the model has 2,457,600 trainable parameters, all of which are part of the LoRA-adapted "query_key_value" modules across the 30 layers of the model. The rest of the model's parameters remain frozen and are not trainable, contributing to the efficiency of this adaptation method.


#Therefore: trainable params: 2457600 || all params: 3005015040
```



In [12]:
from peft import LoraConfig, get_peft_model
config=LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model=get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 2457600 || all params: 3005015040


### Dataset : Version 2 of the Stanford Question Answering Dataset (SQuAD v2),



```
SQuAD v2 Dataset: SQuAD version 2 includes over 100,000 question-answer pairs on more than 500 articles. It adds unanswerable questions to the first version, making it more challenging and realistic.

```



In [13]:
from datasets import load_dataset
qa_dataset=load_dataset("squad_v2")

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

### Map dataset



```
#The mapped_qa_dataset = qa_dataset.map(...) line is using the map function of the datasets library to apply a transformation to each example in the qa_dataset.
#The transformation is defined by the lambda function, which calls tokenizer(create_prompt(samples['context'], samples['question'], samples['answers'])) on each example.

#For each example in qa_dataset, the map function extracts samples['context'], samples['question'], and samples['answers'] as inputs for the create_prompt function.
```



In [19]:
def create_prompt(_ctx,_q,_a):
  if len(_a["text"])<1:
    _a="No result found"
  else:
    _a=_a["text"][0]
  pmt_tmplt=f'Context:\n{_ctx}\n\n Question:\n{_q}\n\nAnswer:\n{_a}'
  return pmt_tmplt
mapped_qa_dataset = qa_dataset.map(lambda samples: tokenizer(create_prompt(samples['context'], samples['question'], samples['answers'])))

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [32]:
import transformers
trainer=transformers.Trainer(
    model=model,
    train_dataset=mapped_qa_dataset['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        warmup_steps=50,
        max_steps=50,
        learning_rate=1e-3,
        fp16=True,
        output_dir='./',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

max_steps is given, it will override any value given in num_train_epochs


In [33]:
model.config.use_cache = False
trainer.train()

AssertionError: No inf checks were recorded for this optimizer.

## PUSH TO HF

## Inference

In [ ]:
from IPython.display import display, Markdown

def make_inference(context, question):
  batch = tokenizer(f"Context:\n{context}\n\nQuestion:\n{question}\n\nAnswer:\n", return_tensors='pt')

  with torch.cuda.amp.autocast():
    output_tokens = qa_model.generate(**batch, max_new_tokens=200)

  display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))


In [ ]:
context = "LLM fine tuninig method LoRA."
question = "Explain LoRA step by step"

make_inference(context, question)